
# Záróvizsga igazságossága

### Demo

In [3]:
import pandas as pd
import numpy as np

Kiindulásképp azt gondoltam át, milyen adatokat érdemes kinyerni egy kész beosztásból. Az alábbiakat hasznosnak tartottam: 
 1. hány tárgyat tanít egy tanár   
 2. az egyes tárgyakhoz hány tanár és hallgató jut -> átlagos hallgatók száma / tanár
 3. vizsgáztatóként hányszor van beosztva (ennek számolja alapból)
 4. elnökként hányszor van beosztva (ha nem vizsgázató)
 5. tagként hányszor van beosztva (ha nem elnök)
 
(*Alternatív megoldás a 3-5. pontokra:*  egy tanár egy beosztását elképzelhetjük egy 3 elemű vektorként, pl. `[1 0 1]` jelentheti, hogy a tanár vizsgáztatóként és tagként volt jelen. Én végül nem így dolgoztam fel a bemeneti fájlt, így arra a kérdésre nem tud a modellem válaszolni, hogy hányszor volt egy tanár egyszerre vizsgáztató és elnök. Azért gondolom, hogy ez felesleges lehet, mert a munkateher meghatározásához nem kellene kétszer beleszámolni ugyanazt a vizsgaalkalmat. Ha az igazságosság szempontjából mégis úgy döntünk, hogy munkateher szempontjából eltérő egy `[1 0 0]` és egy `[1 1 0]` eset, akkor a fájl feldolgozásakor ezt is tárolni kell.) 

Következő lépésben felállítottam egy példa adathalmazt, melyben az egyes oktatókhoz az alábbi értékeket rendeltem: 
 - hány vizsgán vesznek részt vizsgáztatóként
 - hány tárgyat tanítanak
 - tárgyakra bontva a vizsgázóik száma
 - tárgyakra bontva a hallgatóik száma
Pl. az alábbi számok jellemzik 3 tanár tárgyainak és vizsgázatott hallgatóinak számát. 

In [ ]:
A = {"examines": 34, "total_courses": 3, "exams_per_course": {1: 8, 2: 10, 3: 16}, "students_per_course": {1: 8, 2: 10, 3: 8}}
B = {"examines": 17, "total_courses": 1, "exams_per_course": {0: 17}, "students_per_course": {0: 17}}
C = {"examines": 4, "total_courses": 2, "exams_per_course": {3: 0, 4: 4}, "students_per_course": {3: 8, 4: 4}}

df = pd.DataFrame([A, B, C], index = ["A", "B", "C"])
df["spc_sum"] = df.apply(lambda x: sum(x.students_per_course.values()), axis = 1)
df

Alább pedig az egyes tárgyakról található példaadat:

In [ ]:
Courses = {0: {"students": 17, "teachers": 1}, 
           1: {"students": 8, "teachers": 1 }, 
           2: {"students": 10, "teachers": 1}, 
           3: {"students": 16, "teachers": 2}, 
           4: {"students": 4, "teachers": 1}}

Talán érezhető, hogy a "C" tanár az igazságosabbnál kevesebb vizsgára került beosztásra. 
A példa adathalmazban az `exams_per_course` és `students_per_course` oszlopok nem túl elegánsak, ráadásul az utóbbi még redundáns információkat is tartalmaz. Mindkettő igazából csak bizonyos másik értékek meghatározásához kell. Az alábbi modellben így ezeket az oszlopokat elhagytam. 

#### Problémafelvetés

A modellnek erőteljesen zárt világ feltételezése van. Ha egy tanár pl. más tárgyakat is tanít, de ez nem derül ki a beosztásból, ez a tény nem lesz figyelembe véve. Ugyanígy, csak akkor mondhatjuk bizonyossággal, hogy egy tanár betölthet pl. elnöki pozíciót, ha van olyan beosztás, ahol elnök. 
> **Javaslat:** A bemenetből az alábbi kérdésekre biztosabb választ találni. (Pl. az elérhetőségek lapon megtalálható, hogy ki lehet elnök/tag/titkár.)

Előfordulhat, hogy a modell aránytalanságokat talál, de a valóságban ez kívánt viselkedés. Pl. egy tárgyvezető nagyobb százalékban kíván jelen lenni a vizsgákon, mint a többi oktató. Ha a beosztás is ezt figyelembe véve lett elkészítve, akkor az igazságosság megállapításakor pl. súlyokkal tudjuk ezt az információt kezelni. Az általam implementált modell ezzel nem foglalkozik.

### Input feldolgozása

In [1]:
from collections import defaultdict

In [107]:
class ScheduleStats:
    
    _SCHEDULE_DATA_PATH = './data/Beosztáshoz2020osz.xlsx'
    
    def __init__(self):
        self._schedule = self.load_in_sample_data()
        # The function below presupposes the initialized schedule dataframe
        self.courses = self._init_courses()
        # The function below presupposes the initialized schedule and courses dataframes 
        self.teachers = self._init_teachers()
        
    def load_in_sample_data(self):
        path = ScheduleStats._SCHEDULE_DATA_PATH
        sheet_name = '1.kör'
        usecols = "I:K,M:O,Q"
        
        schedule = self._load_in_schedule(path, sheet_name, usecols)
        
        # resolve NaN due to merged cells        
        schedule[['Elnök', 'Tag', 'Titkár']] = schedule[['Elnök', 'Tag', 'Titkár']].fillna(method = 'ffill')
        
        # filter out rows where student name is NaN        
        schedule = schedule.dropna(subset = ['Név'])
        return schedule
        
    def _load_in_schedule(self, path, sheet_name = None, usecols = None):            
        return pd.read_excel(path, sheet_name = sheet_name, usecols = usecols)           
    
    def _init_courses(self):
        if self._schedule is None:
            raise Exception('Parsing course information was called before initializing a "Schedule" DataFrame to work with. ')
        course_data = [
            {"Tárgy": targy, 
             "Hallgatók": len(self._schedule[self._schedule["Vizsgatárgy"] == targy]),
             "Tanárok": len(self._schedule[self._schedule["Vizsgatárgy"] == targy].Vizsgáztató.unique())}
            for targy in self._schedule.Vizsgatárgy.unique()
        ]
        courses = pd.DataFrame(course_data)
        # Egy tanárra jutó hallgatók száma
        courses['ETJH'] = courses.Hallgatók / courses.Tanárok
        courses.ETJH = courses.ETJH.round().astype(int)
        return courses
    
    def _init_teachers(self, workload_weights = np.ones((4))):
        if self._schedule is None:
            raise Exception('Parsing teacher information was called before initializing a "Schedule" DataFrame to work with. ')
        if self.courses is None:
            raise Exception('')
        all_teachers = pd.unique(self._schedule[["Vizsgáztató", "Elnök", "Tag", "Titkár"]].values.ravel('K'))
        teacher_data = list(defaultdict())
        for teacher in all_teachers:
            exam_count = len(self._schedule[self._schedule.Vizsgáztató == teacher])
            pres_count = len(self._schedule[(self._schedule.Vizsgáztató != teacher) & (self._schedule.Elnök == teacher)])
            mem_count = len(self._schedule[(self._schedule.Vizsgáztató != teacher) & (self._schedule.Tag == teacher)])
            sec_count = len(self._schedule[(self._schedule.Vizsgáztató != teacher) & (self._schedule.Titkár == teacher)])
            taught_courses = self._schedule[self._schedule.Vizsgáztató == teacher].Vizsgatárgy.unique()
            # Taught courses index // (oktatott tárgyak index)
            TCI = len(taught_courses)
            # Cumulative student index (for the teached subjects) // (kumulatív hallgatók index)
            CSI = sum(self.courses[self.courses.Tárgy.isin(taught_courses)].Hallgatók)
            # Average students index (per teacher) // (átlagos hallgatók index)
            ASI = sum(self.courses[self.courses.Tárgy.isin(taught_courses)].ETJH)
            teacher_dict = {"Név": teacher, 
             "Vizsga": exam_count, 
             "Elnök": pres_count,
             "Tag": mem_count,
             "Titkár": sec_count,
             "OTI": TCI,
             "KHI": CSI,
             "ÁHI": ASI,
            }
            teacher_data.append(teacher_dict)
        return pd.DataFrame(teacher_data) 
    
    def select_subset(name): 
        subset_loc = {
            "vizsgáztatók": lambda df: (df["Vizsga"] > 0),
            "egyéb": lambda df: df["Vizsga"] <= 0,
        }

        if name.lower() not in subset_loc.keys():
            raise Exception(f'Subset "{name}" does not exist.')

        return subset_loc[name.lower()]

    def subset(teacher_df, name): 
        return teacher_df[ScheduleStats.select_subset(name)]

    def workload(teacher_df, workload_weights = np.ones(4)):
        col_subset = ["Vizsga", "Elnök", "Tag", "Titkár"]
        wl = teacher_df[col_subset].apply(lambda df: np.dot(df, workload_weights).item(), axis = 1)   
        wl.name = "Munka"
        return wl

        

#### Az osztály használata: 

In [ ]:
stats = ScheduleStats()
display("Schedule", stats._schedule)
display("Courses", stats.courses)
display("Teachers", stats.teachers)

### Regresszió

Az alábbi fejezetben a sklearn regressziós modelljeit vetem össze. Ezen belül is megvizsgálom a lineáris regressziót, a polinomiális regressziót és a Support Vector regressziót. A determinációs együtthatón keresztül vizsgálom, hogy a modellek mennyire illeszkednek a beosztásból kinyert adatokhoz. 
A vizsgált dimenziók az oktatók oktatott tárgyainak száma (OTI), kumulált hallgatóinak száma (KHI) és átlagos hallgatóinak száma (ÁHI). Csak azokat az oktatókat vizsgálom most, akiknek ezen értékek legalább egyike nem 0. (A titkárok és a vizsgáztatók diszkrét halmazokat alkotnak a leírt feltétel szerint, a titkárok egyáltalán nem jelennek meg vizsgáztatói szerepben, így a vizsgált indexek minden esetben nullát vesznek fel az esetükben.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import svm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

A szükséges importálások után előállítom a modell bemeneti és kimeneti értékeit a feljebb kifejtett indexekre való projekcióval. 

In [6]:
stats = ScheduleStats()
input_columns = ['OTI', 'KHI', 'ÁHI']
data = stats.teachers[(stats.teachers[input_columns] != 0).all(axis = 1)]

X, y = data[input_columns], ScheduleStats.workload(data, np.ones(4))

Első körben a lineáris regressziót figyelem. 

In [7]:
model = LinearRegression().fit(X,y)
print("R\u00b2 = ", model.score(X,y))

R² =  0.3582615792721434


Ezt követően a polinomiális regressziót vizsgálom. Ez lényegesen jobb eredményt ér el. 
A később vizsgált modellek ennél gyengébb determinációs együtthatót érnek el, így egyúttal egy táblázatban demonstrálom azt is, hogy az adott bemeneti és kimeneti értékekhez a modell milyen munkaterhet jósol egy oktatónak. 

In [ ]:
model = make_pipeline(PolynomialFeatures(degree = 2, include_bias = False), LinearRegression()).fit(X, y)
print("R\u00b2 = ", model.score(X, y))

y_pred = model.predict(X)
print("Prediction: ")

np_pred = np.array((X.OTI, X.KHI, X.ÁHI, y, y_pred.round(2)))
pd.DataFrame(np_pred.T, columns = ["OTI", "KHI", "ÁHI", "Actual workload", "Prediction"])

A szemléltetés érdekében az alábbi két diagramot rajzoltatom ki. Természetesen egy 4 dimenziós modellt 3 dimenzióban nem lehet teljesen megérteni, ezzel indoklom a nagyon furcsa regressziós görbét.

In [ ]:
plt.scatter(X.ÁHI, y)
plt.plot(X.ÁHI, y_pred, 'r')
plt.ylabel('some numbers')
plt.show()

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(X.KHI, X.ÁHI, y);
ax.plot(X.KHI, X.ÁHI, y_pred, "gray")

Végül a Support Vector Machines alá tartozó Support Vector Regression osztályt vizsgáltam. Ezen belül három kernel módot próbáltam ki, ezek a Radial basis function, Lineáris és Polinom. Mindhárom módot skálázással és anélkül is vizsgáltam.
Míg az RBF esetében a skálázás rontott az eredményen, a Polinomnál 10%-ot javított. (A lineáris módnál nem változott, hiszen a skálázás maga is egy lineáris művelet.) Az RBF (skálázás nélkül) érte el a legmagasabb determinációs együttható értéket (0.3689). 

In [ ]:
svr_rbf = svm.SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = svm.SVR(kernel="linear", C=100, gamma="auto")
svr_poly = svm.SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
models = [("RBF", svr_rbf), ("LIN", svr_lin), ("POLY", svr_poly)]
print("Models without scaling: ")
for name, model in models:
    model.fit(X, y)
    print("R\u00b2 of %s = %s" % (name, model.score(X, y)))
    
print("Models with scaling: ")
for name, model in models:
    ppl = make_pipeline(StandardScaler(), model)
    ppl.fit(X, y)
    print("R\u00b2 of %s = %s" %(name, ppl.score(X, y)))

### Igazságosság

Most, hogy elő tudjuk állítani a releváns bemeneti adatokat és van némi tapasztalatunk pár regressziós modellel, ideje megválaszolni az eredeti kérdést: igazságos-e a beosztás. 

In [8]:
import os, pickle
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [182]:
class FairnessChecker:
    
    SAVED_DATASET = './data/schedule_dataset.pickle'
    
    def _load_pickled(self, path):
        with open(path, mode = 'rb') as f:
            return pickle.load(f)
        
    def _pickle_obj(self, obj, path):
        with open(path, mode = 'wb') as f:
            pickle.dump(obj, f)
        
    def _init_dataset(self, force_reload):
        path = FairnessChecker.SAVED_DATASET
        
        if os.path.exists(path) and not force_reload:
            self.dataset = self._load_pickled(path)
        else:
            self.dataset = ScheduleStats()
            self._pickle_obj(self.dataset, path)        
            
    def default_model(): 
        default_poly_degree = 2
        pipeline = Pipeline([
            ("polynomial_features", PolynomialFeatures(degree = default_poly_degree, include_bias = False)), 
            ("linear_regression", LinearRegression()),
        ])
        return pipeline
    
    def __init__(self, model = default_model, force_reload = False):
        if callable(model):
            self.model = model()
        else:
            self.model = model
        self._init_dataset(force_reload)
        
    def train_test(self, 
                   select = True, 
                   project = ["Elnök", "Tag", "Titkár", "OTI", "KHI", "ÁHI"], 
                   aggregate = None,
                   workload_weights = np.ones(4)):
        df = self.dataset.teachers
        if aggregate:
            aggr_from, aggr_to, aggr_lam = aggregate
            df[aggr_to] = df[aggr_from].apply(aggr_lam, axis = 1)
        df = df[select]
        wl = ScheduleStats.workload(df, workload_weights)
        return df[project], wl
    
    def fit(self, X, y):
        self.model.fit(X, y)
        
    def predict(self, X):
        return self.model.predict(X)
    
    def _examiner_fair_check():
        checker = FairnessChecker()    
        teacher_selection = ScheduleStats.select_subset("vizsgáztatók")
        X, y = checker.train_test(
            select = teacher_selection,
            project = ["Extra", "OTI", "KHI", "ÁHI"],
            aggregate = (["Elnök", "Tag", "Titkár"], "Extra", lambda df: df.sum()),
            workload_weights = (1., 0., 0., 0.),
        )
        checker.fit(X, y)
        y_pred = pd.Series(checker.predict(X).round(2), name = "Pred")
        return pd.concat([y, y_pred], axis = 1)
        
    def _others_fair_check(checker = FairnessChecker()):
        X = ScheduleStats.subset(checker.dataset.teachers, "egyéb")
        y = ScheduleStats.workload(X, workload_weights = (0, 1, 1, 1))
        y_pred = pd.Series(round(np.mean(y),2), index = y.index, name = "Pred")
        return pd.concat([y, y_pred], axis = 1)
    
    def fair_check(threshold = 1.0):
        checker = FairnessChecker()

        examiners_df = FairnessChecker._examiner_fair_check()
        others_df = FairnessChecker._others_fair_check(checker)
        fairness_df = examiners_df.append(others_df)
        fairness_df = pd.concat([checker.dataset.teachers["Név"], fairness_df], axis = 1)

        scaler = StandardScaler(with_mean = False)
        fairness_df["Std_Pred"] = (fairness_df.Munka - fairness_df.Pred)**2
        fairness_df["Std_Pred"] = scaler.fit_transform(fairness_df["Std_Pred"].values.reshape((-1,1))).round(2)
        return fairness_df[fairness_df.Std_Pred > 1]

In [183]:
unfair_results = FairnessChecker.fair_check()
if unfair_results.empty: 
    print("A beosztás a modell szerint igazságos! :-) ")
else:
    print("A beosztás a modell szerint az alábbi oktatók esetén nem volt igazságos:  ")
    display(unfair_results)

A beosztás a modell szerint az alábbi oktatók esetén nem volt igazságos:  


,Név,Munka,Pred,Std_Pred
0,Dr. Dudás Ákos,23.0,13.88,2.76
1,Imre Gábor,6.0,13.88,2.06
22,Dr. Kovács Tibor,21.0,10.39,3.74
30,Kovács László,1.0,10.39,2.93
33,Dr. Nagy Ákos,4.0,10.39,1.36
35,Sik Tamás Dávid,21.0,10.39,3.74
36,Veréb Szabolcs,16.0,10.39,1.04


## Lehetséges fejlesztési irányok

Feltehetően nem lesz ugyanolyan sémájú az összes bemeneti excel fájlunk. `A ScheduleStats` osztály jelen formájában egy konkrét bemeneti fájlból indul ki. Érdemes lehet a beolvasott fájl feldolgozását lambda függvényeken keresztül elvégezni és a mostani, fájlspecifikus feldolgozást alapértelmezettként kezelni. Alternatíva lehet egy leszármazott osztályban implementálni az összes, a 2020-as bemeneti Excel fájlra jellemző feldolgozást, így növelve a kód újrafelhasználhatóságát.  

A kód követhetősége érdekében érdemes lehet áttérni mindenhol az angol terminológiára. A DataFramek oszlopnevei egyelőre szimplán átveszik a beolvasott fájl fejléceit, így a kódban is sok helyen zavaró lehet a magyar. (Másrészről viszont igaz, hogy a kód teljes egészében a BME-VIK záróvizsgabeosztásainak igazságosságával foglalkozik, helyénvaló lehet meghagyni változatlanul az oszlopokat.)